In [1]:
import tensorflow as tf
import keras
import os
import shutil
import cv2
import numpy as np
import glob as glob
import matplotlib.pyplot as plt
from keras.models import load_model

In [3]:
print(len(glob.glob("Evaluation/Test/**/*.jpg")))
print(len(glob.glob("Evaluation/Validation/**/*.jpg")))

394
917


In [4]:
train_dir = "Training"
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    interpolation='area'
)

valid_dir = "Evaluation/Validation"
val_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    interpolation='area'
)

Found 5712 files belonging to 4 classes.
Found 917 files belonging to 4 classes.


In [5]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


# Load ResNet 152


In [10]:
model = tf.keras.Sequential()

vgg19_base = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)

for layer in vgg19_base.layers:
    layer.trainable=False


model.add(vgg19_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

In [11]:
import segmentation_models_3D as sm
focal_loss = sm.losses.CategoricalFocalLoss()

In [12]:
epochs = 15
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint =  tf.keras.callbacks.ModelCheckpoint(filepath ='model_checkpoints/vgg19_cc_loss.hdf5',
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max'
                                                 )

# Define some callbacks to improve training.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)


In [14]:
history_vgg19 = model.fit(train_ds,
                          validation_data=val_ds,
                          epochs=epochs,
                          callbacks=[early_stopping, reduce_lr, checkpoint])

Epoch 1/15


357/357 [==============================] - ETA: 0s - loss: 0.4879 - accuracy: 0.8223
Epoch 1: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 152s 424ms/step - loss: 0.4879 - accuracy: 0.8223 - val_loss: 0.3874 - val_accuracy: 0.8451 - lr: 1.0000e-04
Epoch 2/15
357/357 [==============================] - ETA: 0s - loss: 0.2561 - accuracy: 0.9112
Epoch 2: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 149s 416ms/step - loss: 0.2561 - accuracy: 0.9112 - val_loss: 0.3143 - val_accuracy: 0.8735 - lr: 1.0000e-04
Epoch 3/15
357/357 [==============================] - ETA: 0s - loss: 0.1964 - accuracy: 0.9338
Epoch 3: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 148s 414ms/step - loss: 0.1964 - accuracy: 0.9338 - val_loss: 0.2524 - val_accuracy: 0.9084 - lr: 1.0000e-04
Epoch 4/15
357/357 [==============================] - ETA: 0s - loss: 0.1556 - ac

In [15]:
vgg19_base.trainable = True
for layer in vgg19_base.layers[:10]:
    layer.trainable=False
    
fine_tune_history_vgg19 = model.fit(train_ds,
                          validation_data=val_ds,
                          epochs=epochs-5,
                          callbacks=[early_stopping, reduce_lr, checkpoint])

Epoch 1/10
357/357 [==============================] - ETA: 0s - loss: 0.0184 - accuracy: 0.9979
Epoch 1: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 70s 195ms/step - loss: 0.0184 - accuracy: 0.9979 - val_loss: 0.1000 - val_accuracy: 0.9640 - lr: 1.0000e-04
Epoch 2/10
357/357 [==============================] - ETA: 0s - loss: 0.0145 - accuracy: 0.9988
Epoch 2: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 70s 197ms/step - loss: 0.0145 - accuracy: 0.9988 - val_loss: 0.0958 - val_accuracy: 0.9673 - lr: 1.0000e-04
Epoch 3/10
357/357 [==============================] - ETA: 0s - loss: 0.0116 - accuracy: 0.9989
Epoch 3: saving model to model_checkpoints\vgg19_cc_loss.hdf5
357/357 [==============================] - 71s 199ms/step - loss: 0.0116 - accuracy: 0.9989 - val_loss: 0.0905 - val_accuracy: 0.9640 - lr: 1.0000e-04
Epoch 4/10
357/357 [==============================] - ETA: 0s - loss: 0.0

In [16]:
test_dir = "Evaluation/Test"
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    interpolation='area'
)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
score = model.evaluate(test_ds)

Found 394 files belonging to 4 classes.
25/25 [==============================] - 7s 278ms/step - loss: 0.0439 - accuracy: 0.9848


In [ ]:
history = fine_tune_history_resnet50
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(5,8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,0.1])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()